In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NN(nn.Module):
    def __init__(self, input_dim, hidden_dim=32, output_dim=1):
        super().__init__()
        self.layer1 = nn.Linear(input_dim, hidden_dim)
        self.layer2 = nn.Linear(hidden_dim, hidden_dim)
        self.layer3 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.layer3(x)
        return x

In [2]:
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

def calc_accuracy(code,X_train, y_train, X_val, y_val, device=None, hidden_dim=32, batch_size=16, max_epochs=5, lr=0.01):

    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    selected_features = X_train.columns[code]
    if len(selected_features) == 0: #ovo bolje u ga
        return -float('inf')

    X_tr = torch.tensor(X_train[selected_features].to_numpy(),dtype = torch.float32)
    y_tr = torch.tensor(y_train.to_numpy(),dtype = torch.float32)
    
    X_va = torch.tensor(X_val[selected_features].to_numpy(),dtype = torch.float32)
    y_va = torch.tensor(y_val.to_numpy(),dtype = torch.float32)
    
    
    train_dataset = TensorDataset(X_tr, y_tr)
    val_dataset   = TensorDataset(X_va, y_va)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader   = DataLoader(val_dataset,   batch_size=batch_size*2, shuffle=False)

    model     = NN(input_dim=len(selected_features), hidden_dim=hidden_dim).to(device)
    loss_fn   = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for _ in range(max_epochs):
        model.train()
        for X, y in train_loader:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            logits = model(X)
            loss   = loss_fn(logits, y)
            loss.backward()
            optimizer.step()

    model.eval()
    num_correct, num_samples = 0, 0
    with torch.no_grad():
        for X, y in val_loader:
            X, y = X.to(device), y.to(device)
            logits   = model(X)
            preds    = (logits > 0)
            num_correct += (preds == y).sum().item()
            num_samples += y.size(0)
            

    val_acc = num_correct / num_samples
    #print(val_acc)
    return val_acc